In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Web Scraping

In [80]:
#sayfalar arasında gezin ve ilan linklerini topla
url = "https://www.arabam.com/ikinci-el/otomobil?city="
links = []
for j in range(86):
    for i in range(50):
        url1 = url+str(j+1)+"&take=50&page="+str(i+1)
        html = urlopen(url1)
        soup = BeautifulSoup(html, "html.parser")  
        href = [div.a.get('href') for div in soup.find_all(class_='pr10 fade-out-content-wrapper')]
        links.append(href)

In [81]:
# 2boyutlu listeden tek boyuta indirdik
links = [j for sub in links for j in sub]

In [82]:
# aldığım "href" linklerine arabam.com ekleleyerek url yi tamamladık
link = []
for i, value in enumerate(links):
    get_url = "https://www.arabam.com" + value
    link.append(get_url)  
link

['https://www.arabam.com/ilan/galeriden-satilik-renault-clio-1-5-dci-joy/2013-model-clio-1-5-dci-joy-202-000-kmde-temiz/14852779',
 'https://www.arabam.com/ilan/galeriden-satilik-hyundai-accent-blue-1-4-d-cvvt-mode-plus/2017-otomatik-boyasiz-ezik-cizik-leke-yok-sifir-ayarinda-lpg-li-isli/14833557',
 'https://www.arabam.com/ilan/galeriden-satilik-chevrolet-aveo-1-4-sx/aveo-1-4-eco-motor-otomatik-lpg-li-isli-128-binde/14621989',
 'https://www.arabam.com/ilan/galeriden-satilik-volvo-s60-2-0-t-premium/2009-model-ful-l-full-otomatik-vites-premium-paket-sunroouflu/11826398',
 'https://www.arabam.com/ilan/galeriden-satilik-ford-focus-1-6-tdci-collection/2011-model-ford-focus-1-6-tdci-collection-paket-manuel/14715863',
 'https://www.arabam.com/ilan/galeriden-satilik-volkswagen-jetta-1-6-tdi-trendline/kaporasi-alinmistir-yeni-sahibine-hayirli-olsun/14572634',
 'https://www.arabam.com/ilan/galeriden-satilik-mercedes-benz-190-e-2-0/1992-model-otomatik-vites-190e-2-0-firsat-araci/14400493',
 'http

In [108]:
fiyat = []
konum = []
ozellik = []
for i, value in enumerate(link):
    link_html = urlopen(value)
    link_soup = BeautifulSoup(link_html, "html.parser")
    #fiyat
    price = link_soup.find("span", {"class" : "color-red4 font-semi-big bold fl"})
    if price == None:
        price = link_soup.find("span", {"class" : "color-red4 font-semi-big bold fl w66"})
        if price == None:
            fiyat.append("None")
        else:
            fiyat.append(price.text) 
    else:
        fiyat.append(price.text)
    #konum
    location = link_soup.find(class_ = "one-line-overflow font-default-minus pt4 color-black2018 bold")
    if location == None:
        konum.append("None")
    else:
        konum.append(location.text)
    #arac ozellikleri
    info = link_soup.find("ul",{"class":"w100 cf mt16"})
    if info == None:
        ozellik.append("None")
    else:
        ozellik.append(info.text)


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

URLError: <urlopen error [Errno 54] Connection reset by peer>

# Data Preprocessing

In [459]:
#None değerlerini sildik ve indexi düzelttik
df = df[df.Ozellik != 'None'].reset_index()
loc = df["Konum"].str.split("/", n = 1, expand = True)
df.drop(columns=["Konum"], inplace=True)

In [460]:
#Düzenlenmiş "konum" kolonu
loc.drop(columns=[1],inplace=True)

In [461]:
# Araç özelliklerini sütunlara ayırdık
new = df["Ozellik"].str.split(":", n = 14, expand = True)
new

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,İlan No,14852779 İlan Tarihi,02 Temmuz 2020 Marka,Renault Seri,Clio Model,1.5 dCi Joy Yıl,2013 Yakıt Tipi,Dizel Vites Tipi,Düz Motor Hacmi,1461 cc Motor Gücü,75 hp Kilometre,202.000 km Boya-değişen,Belirtilmemiş Takasa Uygun,Takasa Uygun Kimden,Galeriden
1,İlan No,14833557 İlan Tarihi,30 Haziran 2020 Marka,Hyundai Seri,Accent Blue Model,1.4 D-CVVT Mode Plus Yıl,2017 Yakıt Tipi,LPG & Benzin Vites Tipi,Otomatik Motor Hacmi,1368 cc Motor Gücü,100 hp Kilometre,77.000 km Boya-değişen,Belirtilmemiş Takasa Uygun,Takasa Uygun Değil Kimden,Galeriden
2,İlan No,14621989 İlan Tarihi,29 Haziran 2020 Marka,Chevrolet Seri,Aveo Model,1.4 SX Yıl,2009 Yakıt Tipi,LPG & Benzin Vites Tipi,Otomatik Motor Hacmi,1201 - 1400 cm3 Motor Gücü,76 - 100 HP Kilometre,128.000 km Boya-değişen,3 boyalı Takasa Uygun,Takasa Uygun Değil Kimden,Galeriden
3,İlan No,11826398 İlan Tarihi,23 Haziran 2020 Marka,Volvo Seri,S60 Model,2.0 T Premium Yıl,2009 Yakıt Tipi,LPG & Benzin Vites Tipi,Yarı Otomatik Motor Hacmi,1984 cc Motor Gücü,180 hp Kilometre,223.000 km Boya-değişen,Belirtilmemiş Ekspertiz sonucu,İyi Durumda Takasa Uygun,Takasa Uygun Kimden: Galeriden
4,İlan No,14715863 İlan Tarihi,19 Haziran 2020 Marka,Ford Seri,Focus Model,1.6 TDCi Collection Yıl,2011 Yakıt Tipi,Dizel Vites Tipi,Düz Motor Hacmi,1560 cc Motor Gücü,109 hp Kilometre,262.000 km Boya-değişen,Tamamı orjinal Takasa Uygun,Takasa Uygun Kimden,Galeriden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32783,İlan No,14916246 İlan Tarihi,09 Temmuz 2020 Marka,Toyota Seri,Corolla Model,1.33 Life Yıl,2017 Yakıt Tipi,LPG & Benzin Vites Tipi,Düz Motor Hacmi,1329 cc Motor Gücü,99 hp Kilometre,19.500 km Boya-değişen,3 boyalı Takasa Uygun,Takasa Uygun Değil Kimden,Sahibinden
32784,İlan No,14915488 İlan Tarihi,09 Temmuz 2020 Marka,Peugeot Seri,306 Model,1.6 Griffe Yıl,1998 Yakıt Tipi,LPG & Benzin Vites Tipi,Düz Motor Hacmi,1587 cc Motor Gücü,90 hp Kilometre,247.000 km Boya-değişen,3 değişen Takasa Uygun,Takasa Uygun Kimden,Sahibinden
32785,İlan No,14916453 İlan Tarihi,09 Temmuz 2020 Marka,Volkswagen Seri,Polo Model,1.4 TDi Comfortline Yıl,2016 Yakıt Tipi,Dizel Vites Tipi,Düz Motor Hacmi,1422 cc Motor Gücü,90 hp Kilometre,44.500 km Boya-değişen,4 boyalı Takasa Uygun,Takasa Uygun Değil Kimden,Sahibinden
32786,İlan No,14915136 İlan Tarihi,09 Temmuz 2020 Marka,Renault Seri,Symbol Model,1.5 dCi Joy Yıl,2015 Yakıt Tipi,Dizel Vites Tipi,Düz Motor Hacmi,1461 cc Motor Gücü,75 hp Kilometre,166.500 km Boya-değişen,"1 değişen, 5 boyalı Takasa Uygun",Takasa Uygun Değil Kimden,Sahibinden


In [462]:
# Sütunlara ayırdığımız özellikleri düzenledik ve düzenledigimiz konum, fiyat bilgilerini ekledik
new["Ilan_no"] = new[1].str.replace('İlan Tarihi', '')
new["Ilan_tarihi"] = new[2].str.replace('Marka', '')
new["Marka"] = new[3].str.replace('Seri', '')
new["Seri"] = new[4].str.replace('Model', '')
new["Model"] = new[5].str.replace('Yıl', '')
new["Yıl"] = new[6].str.replace('Yakıt Tipi', '')
new['Yıl'] = [str(x)[:5] for x in new['Yıl']]
new["Yakıt_tipi"] = new[7].str.replace('Vites Tipi', '')
new["Vites_tipi"] = new[8].str.replace('Motor Hacmi', '')
new["Motor_hacmi"] = new[9].str.replace('Motor Gücü', '')
new["Motor_hacmi"] = new["Motor_hacmi"].str.replace("cc", "")
new["Motor_gücü"] = new[10].str.replace('Kilometre', '')
new['Motor_gücü'] = [str(x)[:3] for x in new['Motor_gücü']] 
new['Motor_hacmi'] = [str(x)[:5] for x in new['Motor_hacmi']]
new["Motor_gücü"] = new["Motor_gücü"].str.replace("hp", "")
new["Kilometre"] = new[11].str.replace('Boya-değişen', '')
new["Kilometre"] = new["Kilometre"].str.replace("km", "")
new["Kilometre"] = new["Kilometre"].str.replace(",", ".")
new["Boya-değişen"] = new[12].str.replace('Takasa Uygun', '')
new["Takasa_uygun"] = new[13].str.replace('Kimden', '')
new["Kimden"] = new[14]
new["Konum"]=loc[0]

new["Fiyat"] = df["Fiyat"].str.replace(".", "")
new["Fiyat"] = new["Fiyat"].str.replace("TL", "")
new["Kilometre"] = new["Kilometre"].str.replace('.', '')


In [463]:
new.drop(columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14], axis=1, inplace=True)
new

,Ilan_no,Ilan_tarihi,Marka,Seri,Model,Yıl,Yakıt_tipi,Vites_tipi,Motor_hacmi,Motor_gücü,Kilometre,Boya-değişen,Takasa_uygun,Kimden,Konum,Fiyat
0,14852779,02 Temmuz 2020,Renault,Clio,1.5 dCi Joy,2013,Dizel,Düz,1461,75,202000,Belirtilmemiş,Takasa Uygun,Galeriden,ADANA,77250
1,14833557,30 Haziran 2020,Hyundai,Accent Blue,1.4 D-CVVT Mode Plus,2017,LPG & Benzin,Otomatik,1368,10,77000,Belirtilmemiş,Takasa Uygun Değil,Galeriden,ADANA,125750
2,14621989,29 Haziran 2020,Chevrolet,Aveo,1.4 SX,2009,LPG & Benzin,Otomatik,1201,76,128000,3 boyalı,Takasa Uygun Değil,Galeriden,ADANA,63000
3,11826398,23 Haziran 2020,Volvo,S60,2.0 T Premium,2009,LPG & Benzin,Yarı Otomatik,1984,18,223000,Belirtilmemiş Ekspertiz sonucu,İyi Durumda Takasa Uygun,Takasa Uygun Kimden: Galeriden,ADANA,75000
4,14715863,19 Haziran 2020,Ford,Focus,1.6 TDCi Collection,2011,Dizel,Düz,1560,10,262000,Tamamı orjinal,Takasa Uygun,Galeriden,ADANA,63950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32783,14916246,09 Temmuz 2020,Toyota,Corolla,1.33 Life,2017,LPG & Benzin,Düz,1329,99,19500,3 boyalı,Takasa Uygun Değil,Sahibinden,İSTANBUL,105000
32784,14915488,09 Temmuz 2020,Peugeot,306,1.6 Griffe,1998,LPG & Benzin,Düz,1587,90,247000,3 değişen,Takasa Uygun,Sahibinden,İSTANBUL,28250
32785,14916453,09 Temmuz 2020,Volkswagen,Polo,1.4 TDi Comfortline,2016,Dizel,Düz,1422,90,44500,4 boyalı,Takasa Uygun Değil,Sahibinden,İSTANBUL,120000
32786,14915136,09 Temmuz 2020,Renault,Symbol,1.5 dCi Joy,2015,Dizel,Düz,1461,75,166500,"1 değişen, 5 boyalı",Takasa Uygun Değil,Sahibinden,İSTANBUL,72000


In [464]:
# 'Object' olan kolonların sağ ve solundaki boşlukları sildik
def strip_col(x):
    if x.dtype == object:
        x = x.str.strip()
    return(x)

new = new.apply(strip_col)

In [465]:
# Target değeri Euro ve Dolar para birimi içerdiği için bu satırları dropladık
patternEUR = "EUR"
patternUSD = "USD"

USD = new['Fiyat'].str.contains(patternUSD)
EUR = new['Fiyat'].str.contains(patternEUR)

new = new[~USD]
new = new[~EUR]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [469]:
#Satırları kaymış ve eksik bilgilere sahip olan kolonları dropladık
new = new[new.Yıl != 'LPG']
new = new.replace(to_replace='None', value=np.nan).dropna()
new = new[new.Yıl != 'Benz'].reset_index()
new.drop(columns=["index"], inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [470]:
new["Motor_gücü"] = new["Motor_gücü"].replace(to_replace='-', value=np.nan)
new["Motor_hacmi"] = new["Motor_hacmi"].replace(to_replace='-', value=np.nan)
new["Motor_gücü"].fillna(new["Motor_gücü"].mode().values[0], inplace = True)
new["Motor_hacmi"].fillna(new["Motor_hacmi"].mode().values[0], inplace = True)

In [ ]:
new["Yıl"] = new["Yıl"].astype(int)
new["Fiyat"] = pd.to_numeric(new["Fiyat"])
new["Kilometre"] = new["Kilometre"].astype(float)
new["Motor_gücü"] = new["Motor_gücü"].astype(int)
new["Motor_hacmi"] = new["Motor_hacmi"].astype(int)

In [456]:
new.to_csv("webdata.csv")

In [457]:
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31937 entries, 0 to 31936
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Ilan_no       31937 non-null  object 
 1   Ilan_tarihi   31937 non-null  object 
 2   Marka         31937 non-null  object 
 3   Seri          31937 non-null  object 
 4   Model         31937 non-null  object 
 5   Yıl           31937 non-null  int64  
 6   Yakıt_tipi    31937 non-null  object 
 7   Vites_tipi    31937 non-null  object 
 8   Motor_hacmi   31937 non-null  int64  
 9   Motor_gücü    31937 non-null  int64  
 10  Kilometre     31937 non-null  float64
 11  Boya-değişen  31937 non-null  object 
 12  Takasa_uygun  31937 non-null  object 
 13  Kimden        31937 non-null  object 
 14  Konum         31937 non-null  object 
 15  Fiyat         31937 non-null  int64  
dtypes: float64(1), int64(4), object(11)
memory usage: 3.9+ MB
